# Classificação e teste de hipóteses: conexão

O teste de hipóteses possui uma **conexão intrínseca** com o conceito de classificação, tanto na estatística tradicional quanto nas abordagens modernas de inteligência computacional e ciência de dados.

Essa conexão manifesta-se em diversos níveis: na formalização dos problemas de decisão, na avaliação da qualidade dos modelos de classificação e na validação dos agrupamentos (que são frequentemente chamados de classificação não supervisionada).

### 1. Formalização do Problema de Decisão e Tipos de Erro

A classificação supervisionada (ou simplesmente classificação) é essencialmente um **problema de decisão**, onde uma classe é escolhida com base nos valores das variáveis de entrada. A teoria de classificação, particularmente a **Classificação Bayesiana**, fornece uma formalização matemática para essa tomada de decisão.

#### Matriz de Confusão e Testes Estatísticos
O resultado de um modelo de classificação é frequentemente resumido por uma **matriz de confusão** (também utilizada para avaliação de classificadores), que é exatamente o instrumento usado na estatística para **representar as possíveis soluções de um teste de hipóteses**.

No contexto de classificação binária (onde há classes positiva e negativa), os erros gerados pelo classificador correspondem diretamente aos erros estatísticos de um teste de hipóteses:

1.  O **Falso Positivo (FP)**, que ocorre quando a classe positiva é predita, mas a classe real é negativa, é conhecido na nomenclatura de testes de hipóteses como **Erro do Tipo I**.
2.  O **Falso Negativo (FN)**, que ocorre quando a classe negativa é predita, mas a classe real é positiva, é conhecido na nomenclatura de testes de hipóteses como **Erro do Tipo II**.

Dessa forma, o problema de classificação está, desde sua formalização estatística, ligado à lógica de testes de hipóteses, especialmente ao considerar a minimização do risco condicional de classificação incorreta.

### 2. Avaliação e Comparação de Classificadores

O teste de hipóteses é uma técnica clássica de estatística que permite avaliar se um conjunto de evidências observadas **confirma ou refuta uma hipótese**. Na ciência de dados, essa técnica é crucial para a **avaliação de classificadores**, especialmente ao comparar o desempenho de diferentes modelos.

Em testes comparativos (como os *testes benchmark*), as hipóteses são formuladas para determinar a significância estatística das diferenças de desempenho entre os modelos:

*   **Hipótese nula ($\mathcal{H}_0$):** Os modelos (classificadores) apresentam resultados equivalentes.
*   **Hipótese alternativa ($\mathcal{H}_1$):** Os modelos não apresentam resultados equivalentes.

Ao realizar esses testes, o objetivo é verificar se a diferença de desempenho observada (por exemplo, na acurácia) entre os classificadores é real ou se é apenas um resultado obtido em função de fatores aleatórios.

#### O Valor-p na Avaliação de Classificadores
O resultado do teste de hipóteses é o **valor-p** (ou $P$-value), que mede a **significância estatística das diferenças** observadas na avaliação.

*   O $P$-value é interpretado como a probabilidade do Erro do Tipo I (falso negativo), ou seja, a probabilidade de rejeitar a hipótese nula quando ela é, de fato, verdadeira.
*   Tipicamente, a hipótese nula ($\mathcal{H}_0$) é rejeitada se $P$-value $< 0.05$, o que representa uma **confiança estatística de 95%** na hipótese alternativa.
*   Por exemplo, um estudo pode concluir, usando o teste de hipóteses, que um modelo apresenta um desempenho melhor que os demais com uma confiança estatística em torno de 92%.

Testes não paramétricos, como o teste de Friedman, são frequentemente recomendados para a avaliação e comparação de classificadores.

### 3. Conexão em Agrupamentos (Classificação Não Supervisionada)

A **análise de agrupamentos** é frequentemente chamada de **classificação não supervisionada**, pois não existe uma variável de saída predefinida (variável alvo). Nesses processos, a **validação dos grupos** formados é crucial, e, por se tratar de um processo automático, alguma medida de validade se faz necessária.

Embora muitos indicadores de validade de agrupamentos sejam não-estatísticos (usando critérios relativos), existem aqueles que **fazem testes estatísticos**. O índice Calinski-Harabasz (CH), por exemplo, é um índice de validade que **faz testes estatísticos** e utiliza apenas a estrutura interna dos dados para validar os agrupamentos.

Em suma, seja na modelagem preditiva supervisionada (classificação, regressão logística, classificação bayesiana) ou na análise descritiva não supervisionada (agrupamentos), o teste de hipóteses fornece o rigor estatístico necessário para formalizar decisões sobre a adequação do modelo ou a significância das diferenças encontradas.

# Classificação com ClickHouse usando SQL: Exemplo Completo com Dataset Iris

O ClickHouse oferece funções de machine learning embutidas que permitem resolver problemas de classificação diretamente usando SQL, sem precisar exportar dados para Python ou R. Você pode treinar modelos, fazer inferências e avaliar resultados tudo dentro do banco de dados, aproveitando o processamento paralelo e a alta performance do ClickHouse.

Este lab demonstra como resolver problemas de classificação multiclasse usando apenas SQL no ClickHouse, aplicado ao dataset Iris clássico de Fisher.

![Diagrama da estratégia One-vs-Rest para classificação multiclasse do dataset Iris](diagrama-class.png)
Diagrama da estratégia One-vs-Rest para classificação multiclasse do dataset Iris

## Fundamentos da Abordagem

O ClickHouse implementa classificação usando a função stochasticLogisticRegression, que realiza classificação binária através de descida de gradiente estocástico. 

*****

Para o dataset Iris com três classes (setosa, versicolor, virginica), aplica-se a estratégia One-vs-Rest (OvR), também conhecida como One-vs-All.​

### Estratégia One-vs-Rest

A abordagem One-vs-Rest decompõe um problema multiclasse em múltiplos problemas binários. Para N classes, cria-se N classificadores binários independentes. Cada classificador aprende a distinguir uma classe específica de todas as outras classes combinadas. No caso do Iris com três classes, são necessários três modelos:​

Modelo 1: setosa (+1) versus [versicolor, virginica] (-1)

Modelo 2: versicolor (+1) versus [setosa, virginica] (-1)

Modelo 3: virginica (+1) versus [setosa, versicolor] (-1)

Durante a inferência, todos os três modelos fazem predições e a classe com a maior probabilidade é escolhida como resultado final.​

## Estrutura do Laboratório

O laboratório está organizado em três arquivos principais que cobrem todo o pipeline de machine learning no ClickHouse.

### Dataset Preparado

O arquivo CSV contém 150 amostras do dataset Iris com quatro features numéricas (comprimento e largura de sépalas e pétalas) e três classes balanceadas (50 amostras cada). Este dataset clássico, criado por Ronald Fisher em 1936, é ideal para demonstrar conceitos de classificação por ser simples mas não trivial.​

## Script SQL Completo

O script implementa o pipeline completo de machine learning em nove etapas estruturadas:

### Etapa 1: Criação da Estrutura

```sql
CREATE DATABASE IF NOT EXISTS lab_ml;
use lab_ml;
DROP TABLE IF EXISTS lab_ml.iris;
CREATE TABLE lab_ml.iris (
    Id UInt32,
    SepalLengthCm Float32,
    SepalWidthCm Float32,
    PetalLengthCm Float32,
    PetalWidthCm Float32,
    Species String
) ENGINE = MergeTree()
ORDER BY Species;

```

A tabela usa o motor MergeTree, que é otimizado para operações analíticas e oferece excelente performance para leitura.​

### Etapa 1.1: Carregamento dos Dados

Todos registros entram na tabela `iris` com a coluna `split` definida como 'train' por padrão.

Download do Dataset Iris e Descompactação.

```bash
curl -L -o iris.zip\
  https://www.kaggle.com/api/v1/datasets/download/uciml/iris
unzip iris.zip
```
Carregamento dos dados para a tabela ClickHouse.

```bash
cd Aula3
clickhouse-client --query "INSERT INTO lab_ml.iris (Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species) FORMAT CSVWithNames" < Iris.csv
clickhouse-client --query "SELECT COUNT(*) FROM lab_ml.iris"
clickhouse-client --query "SELECT Species, avg(PetalLengthCm) AS avg_pl FROM lab_ml.iris GROUP BY Species ORDER BY avg_pl DESC"
```

```sql
alter table lab_ml.iris add column split String DEFAULT 'train';
```

Divisão Principal: 80% Treino / 20% Teste
Use 80% dos dados para treino (120 amostras) e 20% para teste (30 amostras). Esta é a divisão padrão e balanceada para datasets pequenos:​

```sql
-- Adiciona com o valor já calculado
ALTER TABLE lab_ml.iris 
ADD COLUMN split String DEFAULT if(Id % 5 = 0, 'test', 'train');
```

### Etapa 2: Transformação para Classificação Binária

```sql
use lab_ml;
CREATE TABLE IF NOT EXISTS iris_binary (
    id UInt32,
    sepal_length Float32,
    sepal_width Float32,
    petal_length Float32,
    petal_width Float32,
    species String,
    split String,
    label_setosa Int8,
    label_versicolor Int8,
    label_virginica Int8
) ENGINE = MergeTree()
ORDER BY id;

   
INSERT INTO lab_ml.iris_binary
SELECT
    Id, SepalLengthCm, SepalWidthCm, PetalLengthCm, PetalWidthCm,
    Species, split,
    if(Species = 'setosa', 1, -1) as label_setosa,
    if(Species = 'versicolor', 1, -1) as label_versicolor,
    if(Species = 'virginica', 1, -1) as label_virginica
FROM lab_ml.iris;
```

Esta transformação é crucial porque o ClickHouse requer que os labels estejam no intervalo {-1, 1} para classificação binária. O valor +1 representa a classe positiva (de interesse) e -1 representa a classe negativa (resto).​

### Etapa 3: Treinamento dos Modelos

```sql
CREATE OR REPLACE VIEW modelo_setosa AS
SELECT
    stochasticLogisticRegressionState(
        0.1,    -- learning rate
        0.0,    -- L2 regularization
        10,     -- mini-batch size
        'SGD'   -- método de otimização
    )(label_setosa, sepal_length, sepal_width, 
      petal_length, petal_width) AS state
FROM iris_binary
WHERE split = 'train';

CREATE OR REPLACE VIEW modelo_versicolor AS
SELECT
    stochasticLogisticRegressionState(
        0.1,    -- learning rate
        0.0,    -- L2 regularization
        10,     -- mini-batch size
        'SGD'   -- método de otimização
    )(label_versicolor, sepal_length, sepal_width, 
      petal_length, petal_width) AS state
FROM iris_binary
WHERE split = 'train';

CREATE OR REPLACE VIEW modelo_virginica AS
SELECT
    stochasticLogisticRegressionState(
        0.1,    -- learning rate
        0.0,    -- L2 regularization
        10,     -- mini-batch size
        'SGD'   -- método de otimização
    )(label_virginica, sepal_length, sepal_width, 
      petal_length, petal_width) AS state
FROM iris_binary
WHERE split = 'train';
```

A função stochasticLogisticRegressionState aceita quatro hiperparâmetros críticos:​

Learning rate (0.1): controla a velocidade de convergência do gradiente descendente. Valores muito altos podem causar divergência, enquanto valores muito baixos tornam o treinamento lento.​

L2 regularization (0.0): coeficiente de regularização para prevenir overfitting. O valor padrão é 0.1, mas começamos com 0.0 para simplicidade.​

Mini-batch size (10): número de elementos cujos gradientes são computados e somados antes de realizar um passo do gradiente descendente. Batches maiores proporcionam maior estabilidade.​

Método de otimização ('SGD'): algoritmo de atualização dos pesos. Opções incluem Adam (padrão), SGD, Momentum e Nesterov.​

### Etapa 4: Inferência e Predições

**ESTÁ ERRADO**

```sql
CREATE OR REPLACE VIEW predicoes AS
WITH
    (SELECT state FROM modelo_setosa) AS model_setosa,
    (SELECT state FROM modelo_versicolor) AS model_versicolor,
    (SELECT state FROM modelo_virginica) AS model_virginica
SELECT
    id, sepal_length, sepal_width, petal_length, petal_width,
    species AS species_real,
    
    evalMLMethod(model_setosa, sepal_length, sepal_width, 
                 petal_length, petal_width) AS prob_setosa,
    evalMLMethod(model_versicolor, sepal_length, sepal_width, 
                 petal_length, petal_width) AS prob_versicolor,
    evalMLMethod(model_virginica, sepal_length, sepal_width, 
                 petal_length, petal_width) AS prob_virginica,
    
    CASE
        WHEN prob_setosa > prob_versicolor 
             AND prob_setosa > prob_virginica THEN 'setosa'
        WHEN prob_versicolor > prob_setosa 
             AND prob_versicolor > prob_virginica THEN 'versicolor'
        ELSE 'virginica'
    END AS species_predita
FROM iris_binary
WHERE split = 'test';
```

A função evalMLMethod aplica o modelo treinado aos dados, retornando probabilidades. O primeiro argumento é o objeto AggregateFunctionState (o modelo), seguido pelas colunas de features. A predição final usa argmax para selecionar a classe com maior probabilidade.​

### Etapa 5: Avaliação do Modelo

```sql
-- Acurácia geral
SELECT
    round(avg(if(species_real = species_predita, 1.0, 0.0)) * 100, 2) 
    AS acuracia_percentual
FROM predicoes;

-- Matriz de confusão
SELECT
    species_real,
    species_predita,
    COUNT(*) as total
FROM predicoes
GROUP BY species_real, species_predita
ORDER BY species_real, species_predita;
```

A avaliação usa agregações SQL simples para calcular métricas de desempenho. A matriz de confusão revela onde o modelo comete erros específicos entre as classes.​

Guia do Laboratório
O README fornece documentação completa incluindo objetivos de aprendizado, instruções de execução, conceitos teóricos e exercícios propostos em três níveis de dificuldade.

Vantagens da Abordagem SQL
Implementar machine learning diretamente no ClickHouse oferece benefícios significativos para determinados casos de uso:​

Performance e Escala: O motor de execução paralela do ClickHouse processa agregações sobre petabytes de dados em segundos, aproveitando formato de armazenamento colunar. Isso elimina a necessidade de exportar dados para frameworks externos.​

Simplicidade Arquitetural: Todo o pipeline permanece no banco de dados, reduzindo complexidade operacional. Não há necessidade de orquestrar múltiplas ferramentas ou gerenciar movimentação de dados entre sistemas.​

Latência Reduzida: Modelos executam onde os dados residem, eliminando transferência de rede. Para aplicações em tempo real, isso pode ser decisivo.​

Acessibilidade: Analistas e engenheiros de dados podem implementar modelos usando SQL familiar, sem dependências em Python ou R.​

Limitações e Considerações
A abordagem tem restrições importantes que devem ser consideradas:​

Apenas Classificação Binária: O ClickHouse implementa somente regressão logística binária nativamente. Problemas multiclasse requerem estratégias meta como One-vs-Rest ou One-vs-One.​

Modelos Limitados: Apenas regressão linear e logística estocásticas estão disponíveis nativamente. Para algoritmos mais sofisticados (árvores de decisão, redes neurais, gradient boosting), é necessário integração com bibliotecas externas ou usar funções como catboostEvaluate.​

Rigidez dos Hiperparâmetros: As implementações são relativamente inflexíveis comparadas a frameworks dedicados de ML. Opções de ajuste fino são limitadas.​

Adequação para Certos Problemas: A abordagem funciona melhor para problemas lineares ou quase-lineares. Para relações não-lineares complexas, modelos externos podem ser necessários.​

Comparação com One-vs-One
Uma estratégia alternativa para multiclasse é One-vs-One (OvO), que treina um classificador para cada par de classes. Para três classes, seriam necessários três modelos:​​

Modelo 1: setosa vs versicolor

Modelo 2: setosa vs virginica

Modelo 3: versicolor vs virginica

O número de modelos é dado por N×(N-1)/2, onde N é o número de classes. Para três classes: 3×2/2 = 3 modelos. A predição usa votação majoritária.​​

Vantagens do OvO: Cada modelo treina em subconjunto menor dos dados, potencialmente mais rápido para datasets grandes. Menos propenso a desbalanceamento de classes.​

Vantagens do OvR: Requer menos modelos para muitas classes (N vs N×(N-1)/2), mais rápido para treinar. Predição baseada em probabilidades é mais direta.​

Para o dataset Iris com três classes balanceadas, One-vs-Rest é geralmente preferido por sua simplicidade e eficiência.​

Aplicações Práticas
Este padrão de classificação SQL no ClickHouse é particularmente útil em cenários específicos:

Feature Store e Engenharia de Features: ClickHouse pode servir como feature store offline/online, fornecendo features agregadas em tempo real para modelos de produção.​

Classificação em Tempo Real: Para aplicações que precisam classificar eventos em streaming com baixa latência, ter o modelo no banco de dados elimina latência de rede.​

Análise Exploratória de ML: Cientistas de dados podem rapidamente treinar e avaliar modelos lineares durante análise exploratória, sem sair do ambiente SQL.​

Sistemas Embarcados: Para aplicações onde simplicidade e pegada de memória são críticas, um modelo SQL pode ser suficiente.